In [ ]:
corpus = ['the woman is a wise queen',
          'the man is a wise president',
          'she is a pretty woman',
          'he is a strong man',
          'she is still young',
          'he is very old',
          'he is the current president of US',
          'the prince is a son of the king',
          'the princess is a daughter of the king',
          'a prince is a young man',
          'a princess is a young woman']

In [ ]:
# q1) 주석 부분에 function1의 목적을 쓰고, 그 주된 목적에 맞게 function1의 이름 변경하기

# 중심단어의 주변단어를 찾는 함수
def neighbor_words(corpus):    
    sentences = []
    for sentence in corpus:
        sentences.append(sentence.split()) #문장을 단어단위로 쪼개기
        # [['the', 'woman', 'is', 'a', 'wise', 'queen'],
        #  ['the', 'man', 'is', 'a', 'wise', 'president']] : sentences 리스트는 이렇게 생겼다!

    window_size = 2 # q2) s의 의미 : window_size

    data = []
    for sentence in sentences: # 문장마다 for문
        for idx, word in enumerate(sentence): # 한 문장의 단어와 단어의 인덱스
            for neighbor in sentence[max(idx - window_size, 0) : min(idx + window_size, len(sentence)) + 1] :
                # max(idx - s, 0):이웃하는 단어들의 처음 인덱스, min(idx + s, len(sentence)) + 1:이웃하는 단어들의 마지막 인덱스
                if neighbor != word: #중심 단어의 이웃하는 단어들만 추가
                    data.append([word, neighbor]) 
    return data

In [ ]:
import pandas as pd

data = neighbor_words(corpus)
df = pd.DataFrame(data, columns = ['input', 'label'])

In [ ]:
# q3) 주석 부분에 function2의 목적을 쓰고, 그 목적에 맞게 function2의 이름 변경하기

#중복없이 단어 목록을 만드는 함수
def word_set(corpus):
    words = []
    for text in corpus: # 문장마다 for문
        for word in text.split(' '): # 한 문장의 단어마다 for문돌려서 word 리스트에 추가
            words.append(word) 
    words = set(words) #중복을 제거한 단어 집합
    return words

In [ ]:
words = word_set(corpus)

In [ ]:
# q4) 주석 부분에 function3의 목적을 쓰고, 그 목적에 맞게 function3과 d의 이름을 변경하기

# 단어에 번호를 붙여주는 함수({단어:인덱스} 딕셔너리)
def word_numbering(words):
    idx = {}
    for i,word in enumerate(words):
        idx[word] = i
    return idx

In [ ]:
d = word_numbering(words)

In [ ]:
import numpy as np

# q5) 주석 부분에 function4의 목적을 쓰고, 그 목적에 맞게 function4와 ohe의 이름 변경하기

# tokenizing한 하나의 단어를 one-hot encoding하는 함수
# word_idx에는 단어의 인덱스(word_numbering 함수의 values를 받음), dim(차원)은 단어개수(len(words))
def one_hot_encoding(word_index, ONE_HOT_DIM):
    encoding = np.zeros(ONE_HOT_DIM)
    encoding[word_index] = 1
    return encoding

In [ ]:
# q6) Word2Vec을 tensorflow로 구현한 코드에서 ? 부분을 올바르게 채워넣기
# q7) 여기서 구현한 Word2Vec의 아키텍쳐는 CBOW or Skip Gram ?

# Skip Gram이다!

# cross entropy 참고
# https://ratsgo.github.io/deep%20learning/2017/10/02/softmax/
# https://kevinthegrey.tistory.com/123

In [ ]:
import tensorflow as tf

ONE_HOT_DIM = len(words) # 인코딩 차원 = 단어개수(중복x)

X = [] # input word
Y = [] # target word

#모든 중심딘어, 주변단어 쌍을 encoding
for x, y in zip(df['input'], df['label']): #input = 중심단어, label = 주변단어
    X.append(one_hot_encoding(d[x], ONE_HOT_DIM)) #d : 단어를 numbering한 딕셔너리({단어:인덱스}) #d[x] : x의 인덱스(value)
    Y.append(one_hot_encoding(d[y], ONE_HOT_DIM))

# convert X,Y to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# embedding dimension
EMBEDDING_DIM = 2

# hidden layer : represent word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM])) 
b1 = tf.Variable(tf.random_normal([1]))
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM])) # 라벨 인코딩차원으로 나와야함!
b2 = tf.Variable(tf.random_normal([1]))
output = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function : cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(output), axis=[1]))

# training
train = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 10000
for i in range(iteration):
    # input : X_train which is one hot encoded word
    # label : Y_train which is one hot encoded neighbor word
    sess.run(train, feed_dict={x: X_train, y_label: Y_train})
    if i % 2000 == 0:
        print('iteration '+ str(i) +' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

In [ ]:
# the hidden layer (W1 + b1) -> look up table
vectors = sess.run(W1 + b1)
vectors

In [ ]:
sess.close()

In [ ]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for word, x1, x2 in zip(w2v_df['word'], w2v_df['x1'], w2v_df['x2']):
    ax.annotate(word, (x1,x2 ))
    
PADDING = 1.0
x1_axis_min = np.amin(vectors, axis=0)[0] - PADDING
x2_axis_min = np.amin(vectors, axis=0)[1] - PADDING
x1_axis_max = np.amax(vectors, axis=0)[0] + PADDING
x2_axis_max = np.amax(vectors, axis=0)[1] + PADDING
 
plt.xlim(x1_axis_min,x1_axis_max)
plt.ylim(x2_axis_min,x2_axis_max)
plt.rcParams["figure.figsize"] = (10,10)

plt.show()